## Елизаров Константин, MADE-DS-21

In [1]:
import re
import numpy as np
from collections import Counter

In [2]:
np.random.seed(13)

In [3]:
RUS_WAP_PATH = './corpora/WarAndPeace.txt'
RUS_AK_PATH = './corpora/AnnaKarenina.txt'

In [4]:
with open(RUS_WAP_PATH, encoding='utf-8') as f:
    rus_wap = f.read()
    
with open(RUS_AK_PATH, encoding='utf-8') as f:
    rus_ak = f.read()
    
rus_corpora = rus_wap + rus_ak

В качестве тренировочного корпуса используется сконкатенированный текст отрывков из Войны и Мира и Анны Карениной на русском языке.

## 1. Базовый частотный метод по Шерлоку Холмсу

In [5]:
# Предобработка текста (приведение к нижнему регистру, удаление пунктуации и символов-разделителей)
def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[ \n\t\\n]+', ' ', text)
    text = ' '.join(re.findall(r'[а-я ]+', text))
    text = re.sub(r'[ ]+', ' ', text)
    
    return text


# Вычисление частот символов в тексте
def compute_freqs(text: str) -> dict:
    size = len(text)
    freqs_dict = {}
    
    for key, val in Counter(text).items():
        freqs_dict[key] = val / size
        
    return freqs_dict


# Создание словаря для кодирования текста (последовательности символов алфавита ставится
# в соответствие перемешенная случайным образом последовательность символов этого же алфавита)
def get_encode_dict(freqs_dict: dict) -> dict:
    vocabulary = list(freqs_dict.keys())
    shuffled_vocabulary = vocabulary.copy()
    np.random.shuffle(shuffled_vocabulary)
    encode_dict = {}
    
    for true_char, fake_char in zip(vocabulary, shuffled_vocabulary):
        encode_dict[true_char] = fake_char
        
    return encode_dict


# Применение отображения к символам текста
def apply_map_dict(text, map_dict):
    mapped_text = [map_dict.get(c, ' ') for c in text]
    return ''.join(mapped_text)


# Создание словаря для расшифровки текста на основе частотного метода
def get_decode_dict(encoded_freqs_dict: dict, exact_freqs_dict: dict) -> dict:
    sorted_encoded_freqs = dict(sorted(encoded_freqs_dict.items(), key=lambda x: x[1], reverse=True))
    sorted_exact_freqs = dict(sorted(exact_freqs_dict.items(), key=lambda x: x[1], reverse=True))
        
    return dict(list(zip(sorted_encoded_freqs.keys(), sorted_exact_freqs.keys())))


# Подсчет доли верно декодированных символов (посимвольная точность)
def char_accuracy(first_text: str, second_text: str) -> float:
    assert len(first_text) == len(second_text), (
        'Sizes of texts are different.'
    )
    
    accuracy_score = 0
    for chr_1, chr_2 in zip(first_text, second_text):
        if chr_1 == chr_2:
            accuracy_score += 1
            
    return accuracy_score / len(first_text)

In [6]:
preprocessed_rus_corpora = preprocess_text(rus_corpora)
rus_corpora_freqs = compute_freqs(preprocessed_rus_corpora)
encode_dict = get_encode_dict(rus_corpora_freqs)

In [33]:
# Тестовый отрывок
# test_text = r"""Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.\n\nВсе смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме. Положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами, и всеми членами семьи, и домочадцами. Все члены семьи и домочадцы чувствовали, что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой, чем они, члены семьи и домочадцы Облонских. Жена не выходила из своих комнат, мужа третий день не было дома. Дети бегали по всему дому, как потерянные; англичанка поссорилась с экономкой и написала записку приятельнице, прося приискать ей новое место; повар ушел еще вчера со двора, во время самого обеда; черная кухарка и кучер просили расчета.\n\nНа третий день после ссоры князь Степан Аркадьич Облонский – Стива, как его звали в свете, – в обычный час, то есть в восемь часов утра, проснулся не в спальне жены, а в своем кабинете, на сафьянном диване. Он повернул свое полное, выхоленное тело на пружинах дивана, как бы желая опять заснуть надолго, с другой стороны крепко обнял подушку и прижался к ней щекой; но вдруг вскочил, сел на диван и открыл глаза.\n\n«Да, да, как это было? – думал он, вспоминая сон. – Да, как это было? Да! Алабин давал обед в Дармштадте; нет, не в Дармштадте, а что-то американское. Да, но там Дармштадт был в Америке. Да, Алабин давал обед на стеклянных столах, да, – и столы пели: Il mio tesoro и не Il mio tesoro,[32] а что-то лучше, и какие-то маленькие графинчики, и они же женщины», – вспоминал он.\n\nГлаза Степана Аркадьича весело заблестели, и он задумался, улыбаясь. «Да, хорошо было, очень хорошо. Много еще что-то там было отличного, да не скажешь словами и мыслями даже наяву не выразишь». И, заметив полосу света, пробившуюся сбоку одной из суконных стор, он весело скинул ноги с дивана, отыскал ими шитые женой (подарок ко дню рождения в прошлом году), обделанные в золотистый сафьян туфли и по старой, девятилетней привычке, не вставая, потянулся рукой к тому месту, где в спальне у него висел халат. И тут он вспомнил вдруг, как и почему он спит не в спальне жены, а в кабинете; улыбка исчезла с его лица, он сморщил лоб.\n\n«Ах, ах, ах! Ааа!..» – замычал он, вспоминая все, что было. И его воображению представились опять все подробности ссоры с женою, вся безвыходность его положения и мучительнее всего собственная вина его.\n\n«Да! она не простит и не может простить. И всего ужаснее то, что виной всему я, виной я, а не виноват. В этом-то вся драма, – думал он. – Ах, ах, ах!» – приговаривал он с отчаянием, вспоминая самые тяжелые для себя впечатления из этой ссоры.\n\nНеприятнее всего была та первая минута, когда он, вернувшись из театра, веселым и довольным, с огромною грушей для жены в руке, не нашел жены в гостиной; к удивлению, не нашел ее и в кабинете и, наконец, увидал ее в спальне с несчастною, открывшею все, запиской в руке."""
# test_text

# Тестовый отрывок
test_text = r"""Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.\n\nВсе смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме. Положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами, и всеми членами семьи, и домочадцами. Все члены семьи и домочадцы чувствовали, что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой, чем они, члены семьи и домочадцы Облонских. Жена не выходила из своих комнат, мужа третий день не было дома. Дети бегали по всему дому, как потерянные; англичанка поссорилась с экономкой и написала записку приятельнице, прося приискать ей новое место; повар ушел еще вчера со двора, во время самого обеда; черная кухарка и кучер просили расчета."""
test_text

'Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.\\n\\nВсе смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме. Положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами, и всеми членами семьи, и домочадцами. Все члены семьи и домочадцы чувствовали, что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой, чем они, члены семьи и домочадцы Облонских. Жена не выходила из своих комнат, мужа третий день не было дома. Дети бегали по всему дому, как потерянные; англичанка поссорилась с экономкой и написала записку приятельнице, прося приискать ей новое место; повар ушел еще вчера со двора, во время самого обеда; черная кухарка и кучер просили расчета.'

In [34]:
preprocessed_test_text = preprocess_text(test_text)
# Шифровка текста
encoded_test_text = apply_map_dict(preprocessed_test_text, encode_dict)
test_freqs = compute_freqs(encoded_test_text)
decode_dict = get_decode_dict(test_freqs, rus_corpora_freqs)
# Расшифровка текста
decoded_test_text = apply_map_dict(encoded_test_text, decode_dict)

In [35]:
# Предобработанный тестовый текст
preprocessed_test_text

'все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему все смешалось в доме облонских жена узнала что муж был в связи с бывшею в их доме француженкою гувернанткой и объявила мужу что не может жить с ним в одном доме положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами и всеми членами семьи и домочадцами все члены семьи и домочадцы чувствовали что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой чем они члены семьи и домочадцы облонских жена не выходила из своих комнат мужа третий день не было дома дети бегали по всему дому как потерянные англичанка поссорилась с экономкой и написала записку приятельнице прося приискать ей новое место повар ушел еще вчера со двора во время самого обеда черная кухарка и кучер просили расчета'

In [36]:
# Зашифрованный тестовый текст
encoded_test_text

'мь яьшвьпнюмд яь чзюяэрирйюяахлтяъвяахлтвясвйавфяъ ьшвьпнюмвфяь чзфяъ ьшвьпнюмвяэряьмр члямь яьч ывнрьзямяарч яренръьсюияй ъвялжъвнвяшпрячлйяеднямяьмфжюяьяедмы цямяюияарч яухвъклй ъсрцятлм хъвъпсрбяюярегфмюнвячлйляшпряъ ячрй пяйюпзяьяъючямяраърчяарч яэрнрй ъю яопряэхрарнйвнрьзялй япх пюбяа ъзяюячлшюп нзъряшлмьпмрмвнрьзяюяьвчючюяьлэхлтвчюяюямь чюяшн ъвчюяь чзюяюяарчршваквчюямь яшн ъдяь чзюяюяарчршвакдяшлмьпмрмвнюяшпряъ пяьчдьнвямяюияьрйюп нзьпм яюяшпряъвясвйарчяэрьпрфнрчяамрх яьнлшвбъряьры аыю ьфянцаюяерн  яьмфжвъдяч йаляьрербяш чяръюяшн ъдяь чзюяюяарчршвакдяренръьсюияй ъвяъ ямдираюнвяюжяьмрюиясрчъвпячлйвяпх пюбяа ъзяъ яеднряарчвяа пюяе твнюяэрямь чляарчлясвсяэрп хфъъд явътнюшвъсвяэрььрхюнвьзяьяосрърчсрбяюяъвэюьвнвяжвэюьсляэхюфп нзъюк яэхрьфяэхююьсвпзя бяърмр яч ьпряэрмвхялы ня щ ямш хвяьряамрхвямрямх чфяьвчртряре авяш хъвфясливхсвяюяслш хяэхрьюнюяхвьш пв'

In [37]:
# Расшифрованный тестовый текст
decoded_test_text

'лие имаирвнлбе иетян ьочогн кудж са куджа пагкаы сеимаирвнлаы иетяы сеимаирвнла ьо илоетд лие итешавоия л коте озвосипнч геса дхсава мро тдг збв л илыхн и зблшеэ л нч коте щуасюдгеспоэ ждлеусасрпой н озфылнва тдгд мро се тогер гнря и снт л оксот коте ьовогесне цро ьуоковгавоия дге руернй кеся н тдмнревясо мдлирлолавоия н иатнтн идьуджатн н лиетн мвесатн иетян н котомакюатн лие мвесб иетян н котомакюб мдлирлолавн мро сер итбива л нч иогнревяирле н мро са пагкот ьоироывот клоуе ивдмайсо иошекшнеиы вэкн зовее илыхасб тегкд иозой мет осн мвесб иетян н котомакюб озвосипнч геса се лбчокнва нх илонч потсар тдга руернй кеся се збво кота керн зежавн ьо лиетд котд пап ьореуыссбе асжвнмаспа ьоииоунваия и цпосотпой н саьниава хаьнипд ьуныревяснюе ьуоиы ьуннипаря ей солое теиро ьолау дшев еъе лмеуа ио клоуа ло луеты иатожо озека меусаы пдчаупа н пдмеу ьуоинвн уаимера'

In [38]:
accuracy = char_accuracy(preprocessed_test_text, decoded_test_text)
print(f'Посимвольная точность частотного метода на тестовом отрывке: {accuracy:.4f}')

Посимвольная точность частотного метода на тестовом отрывке: 0.4053


## 2 Биграммный частотный метод

In [39]:
# Вычисление частот встречаемости н-грамм в тексте
def compute_ngram_freqs(text: str, ngram: int = 2) -> dict:
    ngram_counter = Counter(text[idx : idx + ngram] for idx in range(len(text) - ngram + 1))
    size = sum(ngram_counter.values())
    freqs_dict = {}
    
    for key, val in ngram_counter.items():
        freqs_dict[key] = val / size
        
    return freqs_dict


# Декодирование биграммным частотным методом
def decode_bigram_text(text: str, decode_dict: dict) -> str:
    key_scores = {key: i for i, key in enumerate(decode_dict.keys())}
    decoded_text = []
    for ind in range(len(text)):
        first_bigram, second_bigram = None, None
        left_border = ind - 1
        if left_border >= 0:
            first_bigram = text[left_border: left_border + 2]
        right_border = ind + 1
        if right_border < len(text):
            second_bigram = text[right_border - 1: right_border + 1]
        if first_bigram is not None and second_bigram is not None:
            if key_scores.get(first_bigram, len(key_scores)) < key_scores.get(second_bigram, len(key_scores)):
                decoded_text.append(decode_dict.get(first_bigram, '  ')[1])
            else:
                decoded_text.append(decode_dict.get(second_bigram, '  ')[0])
        elif first_bigram is None:
                decoded_text.append(decode_dict.get(second_bigram, '  ')[0])
        elif second_bigram is None:
                decoded_text.append(decode_dict.get(first_bigram, '  ')[1])

    return ''.join(decoded_text)

In [40]:
rus_corpora_freqs_bigram = compute_ngram_freqs(preprocessed_rus_corpora, ngram=2)
test_freqs_bigram = compute_ngram_freqs(encoded_test_text, ngram=2)
decode_dict = get_decode_dict(test_freqs_bigram, rus_corpora_freqs_bigram)
decoded_test_text = decode_bigram_text(encoded_test_text, decode_dict)

In [41]:
# Предобработанный тестовый текст
preprocessed_test_text

'все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему все смешалось в доме облонских жена узнала что муж был в связи с бывшею в их доме француженкою гувернанткой и объявила мужу что не может жить с ним в одном доме положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами и всеми членами семьи и домочадцами все члены семьи и домочадцы чувствовали что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой чем они члены семьи и домочадцы облонских жена не выходила из своих комнат мужа третий день не было дома дети бегали по всему дому как потерянные англичанка поссорилась с экономкой и написала записку приятельнице прося приискать ей новое место повар ушел еще вчера со двора во время самого обеда черная кухарка и кучер просили расчета'

In [42]:
# Зашифрованный тестовый текст
encoded_test_text

'мь яьшвьпнюмд яь чзюяэрирйюяахлтяъвяахлтвясвйавфяъ ьшвьпнюмвфяь чзфяъ ьшвьпнюмвяэряьмр члямь яьч ывнрьзямяарч яренръьсюияй ъвялжъвнвяшпрячлйяеднямяьмфжюяьяедмы цямяюияарч яухвъклй ъсрцятлм хъвъпсрбяюярегфмюнвячлйляшпряъ ячрй пяйюпзяьяъючямяраърчяарч яэрнрй ъю яопряэхрарнйвнрьзялй япх пюбяа ъзяюячлшюп нзъряшлмьпмрмвнрьзяюяьвчючюяьлэхлтвчюяюямь чюяшн ъвчюяь чзюяюяарчршваквчюямь яшн ъдяь чзюяюяарчршвакдяшлмьпмрмвнюяшпряъ пяьчдьнвямяюияьрйюп нзьпм яюяшпряъвясвйарчяэрьпрфнрчяамрх яьнлшвбъряьры аыю ьфянцаюяерн  яьмфжвъдяч йаляьрербяш чяръюяшн ъдяь чзюяюяарчршвакдяренръьсюияй ъвяъ ямдираюнвяюжяьмрюиясрчъвпячлйвяпх пюбяа ъзяъ яеднряарчвяа пюяе твнюяэрямь чляарчлясвсяэрп хфъъд явътнюшвъсвяэрььрхюнвьзяьяосрърчсрбяюяъвэюьвнвяжвэюьсляэхюфп нзъюк яэхрьфяэхююьсвпзя бяърмр яч ьпряэрмвхялы ня щ ямш хвяьряамрхвямрямх чфяьвчртряре авяш хъвфясливхсвяюяслш хяэхрьюнюяхвьш пв'

In [43]:
# Расшифрованный тестовый текст
decoded_test_text

'ониа алнокаытиа атло вомого с гмя е с гме д инеоя наалнокаелоа атлоя наалнокаее в а пости пниа  бжл т к п стои о  тты  мрунье тнь  е ил ннииваол па  йро  ваоыеге п ом стои хврехлунезаедтльн ь еузап о о ыйеаое ниив ил н и аау стжеска я тс пробтто стои во тунети ел нвчо кехл т колуи а осйп снек о ниаввиа т нилонопоел т к о   лоло  б чгм ло о пнато ичнь ло  атло о стоаале  ло пни ичнеоа атло о стоаале о илонопоелко ил н нса уьбое п ома егевиа нояи о ил н е д и то воноаж то спо иа е алнт а еегнмдиловпссо аччтиа  йрреорабинва в юп истрото ичнеоа атло о стоаале оро  тты  мрунье  и птмобаое оуа по м дтоь  рниие а осйп снекя и ао   стое сосо аяслко в нпнати стои д ц вовн юю уи реакаалеме во леиаотька сезаттозап о   чинл е дучиы в вчэшвиа етми вчй о вчэиым икоапя епои абно нвоелритедалаыи пкн еа   спове п нп стоа  тал но зте ин ь овдиюдрме о д кн  вчй ако втькосв'

In [44]:
accuracy = char_accuracy(preprocessed_test_text, decoded_test_text)
print(f'Посимвольная точность биграммного частотного метода на тестовом отрывке: {accuracy:.4f}')

Посимвольная точность биграммного частотного метода на тестовом отрывке: 0.1351


Качество расшифровки ухудшилось, так как встречаемых биграмм больше, чем одинарных символов, и частотное соответствие провести сложнее. 

## 3 MCMC - sampling

Для расшифровки текста в данном пункте будет использоваться метод Метрополиса-Гастингса MCMC-семплирования с использованием статистики биграмм (биграммы текста можно рассматривать, как состояния цепи Маркова со своими вероятностями перехода). В данном методе в качестве правдободобия текста будет рассматриваться произведение вероятностей всех биграмм, входящих в данный текст. За вероятность каждой биграммы принимается частота появления этой биграммы в тренировочном корпусе. <br><br>
**Шаги алгоритма:**
1. Инициализация перестановки и подсчет значения логарифма правдободобия для текста расшифрованного начальной перестановкой.
2. Выполнение случайной перестановки двух элементов из текущей перестановки.
3. Подсчет логарифма правдободобия расшифрованного текста для новой перестановки.
4. Сравнение текущего и нового правдободобий по критерию из алгоритма Метрополиса-Гастингса:
$$\frac{P_{new}}{P_{current}} \vee 1$$
Если отношение больше 1, то за текущую принимается новая перестановка. Иначе новая перестановка принимается с вероятностью:
$$P = \exp(\log(P_{new}) - \log(P_{current}))$$
5. Если не превышено количество итераций, то переходим к пункту 2.
6. По окончанию всех итераций текущая перестановка считается результатом семплирования. 

In [45]:
corpora_vocab = list(rus_corpora_freqs.keys())
test_vocab = list(test_freqs.keys())

In [46]:
# Вычисление логарифма правдоподобия текста (логарифм произведения частот соответствующих н-грамм)
def ngram_log_likelyhood(text: str, decode_dict: dict, corpora_freqs: dict, ngram: int = 2) -> float:
    decoded_text = apply_map_dict(text, decode_dict)
    log_likelyhood = 0
    for ind in range(len(decoded_text) - ngram + 1):
        chunk = decoded_text[ind: ind + ngram]
#         Дефолтное значение правдободобия биграммы, отсуствующей в тренировочном корпусе
        likelyhood = corpora_freqs.get(chunk, 1 / (len(text) + len(corpora_freqs) ** ngram)) 
        log_likelyhood += np.log(likelyhood)
        
    return log_likelyhood


# Принятие или отклонение новой перестановки по критерию в алгоритме Метрополиса-Гастингса
def metropolis_hastings_log_accept(log_likelyhood: float, new_log_likelyhood: float) -> bool:
    if new_log_likelyhood > log_likelyhood:
        return True
    else:
        return (np.random.rand() < (np.exp(new_log_likelyhood - log_likelyhood)))
    

# Алгоритм Метрополиса-Гастингса
def metropolis_hastings(encoded_text: str, corpora_freqs: dict, corpora_vocab: list, test_vocab: list, 
                        n_iters: int = 15000, n_attempts: int = 5, ngram: int = 2) -> dict:
    best_map_dict = {}
    best_likelyhood = -np.inf
    
#     Реализуются несколько попыток полного цикла MCMC-семплирования для избежания застреваний в локальных максимумах
    for k in range(n_attempts):
        num_acception = 0
        cur_map_dict = {key: val for key, val in zip(test_vocab, corpora_vocab)}
        cur_likelyhood = ngram_log_likelyhood(encoded_text, cur_map_dict, corpora_freqs, ngram=ngram)
    
        for _ in range(n_iters):
            vocab_values = list(cur_map_dict.values())
            ind1, ind2 = np.random.choice(range(len(vocab_values)), 2, replace=False)
            vocab_values[ind1], vocab_values[ind2] = vocab_values[ind2], vocab_values[ind1]
            new_map_dict = {key: val for key, val in zip(list(cur_map_dict.keys()), vocab_values)}
            new_likelyhood = ngram_log_likelyhood(encoded_text, new_map_dict, corpora_freqs, ngram=ngram)
        
            if metropolis_hastings_log_accept(cur_likelyhood, new_likelyhood):
                cur_map_dict = new_map_dict
                cur_likelyhood = new_likelyhood
                num_acception += 1
            
        if cur_likelyhood > best_likelyhood:
            best_likelyhood = cur_likelyhood
            best_map_dict = cur_map_dict
        print(f'Attempt: {k + 1}, Number of acceptions: {num_acception}, final log-likelyhood: {cur_likelyhood}')
    return best_map_dict

In [47]:
mcmc_map_dict = metropolis_hastings(encoded_test_text, rus_corpora_freqs_bigram, corpora_vocab, test_vocab)

Attempt: 1, Number of acceptions: 93, final log-likelyhood: -4790.179751040747
Attempt: 2, Number of acceptions: 93, final log-likelyhood: -4790.179751040747
Attempt: 3, Number of acceptions: 134, final log-likelyhood: -4990.533374164072
Attempt: 4, Number of acceptions: 91, final log-likelyhood: -4790.179751040747
Attempt: 5, Number of acceptions: 107, final log-likelyhood: -4790.179751040747


In [48]:
decoded_test_text = apply_map_dict(encoded_test_text, mcmc_map_dict)

In [49]:
# Предобработанный тестовый текст
preprocessed_test_text

'все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему все смешалось в доме облонских жена узнала что муж был в связи с бывшею в их доме француженкою гувернанткой и объявила мужу что не может жить с ним в одном доме положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами и всеми членами семьи и домочадцами все члены семьи и домочадцы чувствовали что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой чем они члены семьи и домочадцы облонских жена не выходила из своих комнат мужа третий день не было дома дети бегали по всему дому как потерянные англичанка поссорилась с экономкой и написала записку приятельнице прося приискать ей новое место повар ушел еще вчера со двора во время самого обеда черная кухарка и кучер просили расчета'

In [50]:
# Зашифрованный тестовый текст
encoded_test_text

'мь яьшвьпнюмд яь чзюяэрирйюяахлтяъвяахлтвясвйавфяъ ьшвьпнюмвфяь чзфяъ ьшвьпнюмвяэряьмр члямь яьч ывнрьзямяарч яренръьсюияй ъвялжъвнвяшпрячлйяеднямяьмфжюяьяедмы цямяюияарч яухвъклй ъсрцятлм хъвъпсрбяюярегфмюнвячлйляшпряъ ячрй пяйюпзяьяъючямяраърчяарч яэрнрй ъю яопряэхрарнйвнрьзялй япх пюбяа ъзяюячлшюп нзъряшлмьпмрмвнрьзяюяьвчючюяьлэхлтвчюяюямь чюяшн ъвчюяь чзюяюяарчршваквчюямь яшн ъдяь чзюяюяарчршвакдяшлмьпмрмвнюяшпряъ пяьчдьнвямяюияьрйюп нзьпм яюяшпряъвясвйарчяэрьпрфнрчяамрх яьнлшвбъряьры аыю ьфянцаюяерн  яьмфжвъдяч йаляьрербяш чяръюяшн ъдяь чзюяюяарчршвакдяренръьсюияй ъвяъ ямдираюнвяюжяьмрюиясрчъвпячлйвяпх пюбяа ъзяъ яеднряарчвяа пюяе твнюяэрямь чляарчлясвсяэрп хфъъд явътнюшвъсвяэрььрхюнвьзяьяосрърчсрбяюяъвэюьвнвяжвэюьсляэхюфп нзъюк яэхрьфяэхююьсвпзя бяърмр яч ьпряэрмвхялы ня щ ямш хвяьряамрхвямрямх чфяьвчртряре авяш хъвфясливхсвяюяслш хяэхрьюнюяхвьш пв'

In [51]:
# Расшифрованный тестовый текст
decoded_test_text

'все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему все смешалось в доме облонских жена узнала что муж был в связи с бывшею в их доме француженкою гувернанткой и объявила мужу что не может жить с ним в одном доме положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами и всеми членами семьи и домочадцами все члены семьи и домочадцы чувствовали что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой чем они члены семьи и домочадцы облонских жена не выходила из своих комнат мужа третий день не было дома дети бегали по всему дому как потерянные англичанка поссорилась с экономкой и написала записку приятельнице прося приискать ей новое место повар ушел еще вчера со двора во время самого обеда черная кухарка и кучер просили расчета'

In [52]:
accuracy = char_accuracy(preprocessed_test_text, decoded_test_text)
print(f'Посимвольная точность метода MCMC-семплирования на тестовом отрывке: {accuracy:.4f}')

Посимвольная точность метода MCMC-семплирования на тестовом отрывке: 1.0000


Получилась 100% точность на тестовом отрывке.

## 4 Расшифруйте сообщение

In [53]:
message = """←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"""
message

'←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [54]:
message_freqs = compute_freqs(message)
message_vocab = list(message_freqs.keys())

In [65]:
mcmc_map_dict = metropolis_hastings(message, rus_corpora_freqs_bigram, corpora_vocab, 
                                    message_vocab, n_iters=10000, n_attempts=20)

Attempt: 1, Number of acceptions: 66, final log-likelyhood: -1388.897040076368
Attempt: 2, Number of acceptions: 133, final log-likelyhood: -1390.661992183435
Attempt: 3, Number of acceptions: 151, final log-likelyhood: -1247.2948545281786
Attempt: 4, Number of acceptions: 119, final log-likelyhood: -1391.8898346415874
Attempt: 5, Number of acceptions: 150, final log-likelyhood: -1399.1844788354729
Attempt: 6, Number of acceptions: 94, final log-likelyhood: -1399.809178648099
Attempt: 7, Number of acceptions: 112, final log-likelyhood: -1284.5642928552086
Attempt: 8, Number of acceptions: 130, final log-likelyhood: -1301.813356814227
Attempt: 9, Number of acceptions: 128, final log-likelyhood: -1246.3615534290004
Attempt: 10, Number of acceptions: 146, final log-likelyhood: -1247.7301667941192
Attempt: 11, Number of acceptions: 121, final log-likelyhood: -1466.7264086059383
Attempt: 12, Number of acceptions: 123, final log-likelyhood: -1375.1745093822037
Attempt: 13, Number of acceptio

In [66]:
decoded_message = apply_map_dict(message, mcmc_map_dict)

In [67]:
decoded_message

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса фотя конечно я ничего не обещаж'

In [68]:
etalon = r"""если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать 
скорее всего вы все сделали правильно и получите максимальный балл за последнее 
четвертое задание курса хотя конечно я ничего не обещаю"""
etalon = etalon.replace('\n', '')

In [69]:
print(f'Посимвольная точность распознавания закодированного сообщения: {char_accuracy(decoded_message, etalon):.4f}')

Посимвольная точность распознавания закодированного сообщения: 0.9565


Закодированное сообщение получилось расшифровать с достаточной для чтения точностью.

## 5 Триграммы

In [77]:
rus_corpora_freqs_trigram = compute_ngram_freqs(preprocessed_rus_corpora, ngram=3)

In [78]:
mcmc_map_dict = metropolis_hastings(encoded_test_text, rus_corpora_freqs_trigram, corpora_vocab, test_vocab, 
                                    n_iters=10000, ngram=3, n_attempts=5)

Attempt: 1, Number of acceptions: 113, final log-likelyhood: -6582.719715835094
Attempt: 2, Number of acceptions: 80, final log-likelyhood: -11267.628303140265
Attempt: 3, Number of acceptions: 81, final log-likelyhood: -6582.719715835094
Attempt: 4, Number of acceptions: 83, final log-likelyhood: -6582.719715835094
Attempt: 5, Number of acceptions: 87, final log-likelyhood: -6582.719715835094


In [79]:
decoded_test_text = apply_map_dict(encoded_test_text, mcmc_map_dict)
decoded_test_text

'все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему все смешалось в доме облонских жена узнала что муж был в связи с бывшею в их доме француженкою гувернанткой и объявила мужу что не может жить с ним в одном доме положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами и всеми членами семьи и домочадцами все члены семьи и домочадцы чувствовали что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой чем они члены семьи и домочадцы облонских жена не выходила из своих комнат мужа третий день не было дома дети бегали по всему дому как потерянные англичанка поссорилась с экономкой и написала записку приятельнице прося приискать ей новое место повар ушел еще вчера со двора во время самого обеда черная кухарка и кучер просили расчета'

In [80]:
print(f'Посимвольная точность распознавания закодированного сообщения: {char_accuracy(decoded_test_text, preprocessed_test_text):.4f}')

Посимвольная точность распознавания закодированного сообщения: 1.0000


In [81]:
mcmc_map_dict = metropolis_hastings(message, rus_corpora_freqs_trigram, corpora_vocab, message_vocab, 
                                    n_iters=10000, ngram=3, n_attempts=5)

Attempt: 1, Number of acceptions: 76, final log-likelyhood: -2343.281291518298
Attempt: 2, Number of acceptions: 77, final log-likelyhood: -2616.0096236051395
Attempt: 3, Number of acceptions: 45, final log-likelyhood: -2901.030194024916
Attempt: 4, Number of acceptions: 91, final log-likelyhood: -1750.831999643107
Attempt: 5, Number of acceptions: 72, final log-likelyhood: -2018.9640814809231


In [82]:
decoded_message = apply_map_dict(message, mcmc_map_dict)
decoded_message

'если вы видите нормальный или почти нормальный текст у этого сообжения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса фотя конечно я ничего не обежащ'

In [83]:
print(f'Посимвольная точность распознавания закодированного сообщения: {char_accuracy(decoded_message, etalon):.4f}')

Посимвольная точность распознавания закодированного сообщения: 0.9826


## 4-граммы

In [84]:
rus_corpora_freqs_fourgram = compute_ngram_freqs(preprocessed_rus_corpora, ngram=4)

In [85]:
mcmc_map_dict = metropolis_hastings(encoded_test_text, rus_corpora_freqs_fourgram, corpora_vocab, test_vocab, 
                                    n_iters=10000, ngram=4, n_attempts=5)

Attempt: 1, Number of acceptions: 87, final log-likelyhood: -7981.2702997411925
Attempt: 2, Number of acceptions: 148, final log-likelyhood: -24563.449302641337
Attempt: 3, Number of acceptions: 83, final log-likelyhood: -7981.2702997411925
Attempt: 4, Number of acceptions: 75, final log-likelyhood: -7981.2702997411925
Attempt: 5, Number of acceptions: 111, final log-likelyhood: -24490.047535202866


In [86]:
decoded_test_text = apply_map_dict(encoded_test_text, mcmc_map_dict)
decoded_test_text

'все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему все смешалось в доме облонских жена узнала что муж был в связи с бывшею в их доме француженкою гувернанткой и объявила мужу что не может жить с ним в одном доме положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами и всеми членами семьи и домочадцами все члены семьи и домочадцы чувствовали что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой чем они члены семьи и домочадцы облонских жена не выходила из своих комнат мужа третий день не было дома дети бегали по всему дому как потерянные англичанка поссорилась с экономкой и написала записку приятельнице прося приискать ей новое место повар ушел еще вчера со двора во время самого обеда черная кухарка и кучер просили расчета'

In [87]:
print(f'Посимвольная точность распознавания закодированного сообщения: {char_accuracy(decoded_test_text, preprocessed_test_text):.4f}')

Посимвольная точность распознавания закодированного сообщения: 1.0000


In [88]:
mcmc_map_dict = metropolis_hastings(message, rus_corpora_freqs_fourgram, corpora_vocab, message_vocab, 
                                    n_iters=10000, ngram=4, n_attempts=5)

Attempt: 1, Number of acceptions: 131, final log-likelyhood: -6682.697721049311
Attempt: 2, Number of acceptions: 72, final log-likelyhood: -6482.013092300597
Attempt: 3, Number of acceptions: 84, final log-likelyhood: -2303.435478003974
Attempt: 4, Number of acceptions: 125, final log-likelyhood: -5992.158627986645
Attempt: 5, Number of acceptions: 55, final log-likelyhood: -6581.749731364583


In [89]:
decoded_message = apply_map_dict(message, mcmc_map_dict)
decoded_message

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса фотя конечно я ничего не обещаж'

In [90]:
print(f'Посимвольная точность распознавания закодированного сообщения: {char_accuracy(decoded_message, etalon):.4f}')

Посимвольная точность распознавания закодированного сообщения: 0.9913


При расшифровке длинного текста МСМС-алгоритм показал одинаково хорошее качество на биграммах, триграммах и 4-граммах. Для короткого текста качество декодирования повышается с увеличением размера n-грамм. Качество на тестовом предложении для биграмм - `0.9565`, для триграмм - `0.9826`, для 4-грамм - `0.9913`.

## 6 Применения модели

Данная модель может применяться в биологии и генетике, в частности, для анализа последовательностей ДНК, так как при работе с ДНК трудно или практически невозможно идентифицировать или выровнять участки геномов из-за неизвестных распределений или распределений сложной формы в совокупности с огромными объемами данных. Возможно, подобные модели могут найти свое применение в машинном переводе текстов на разных языках.